**[CNN-01]** 必要なモジュールをインポートして、乱数のシードを設定します。

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20160704)
tf.set_random_seed(20160704)

**[CNN-02]** MNISTのデータセットを用意します。

In [2]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


**[CNN-03]** 1段目の畳み込みフィルターとプーリング層を定義します。

In [3]:
num_filters1 = 32

x = tf.placeholder(tf.float32, [None, 784])
x_image = tf.reshape(x, [-1,28,28,1])

W_conv1 = tf.Variable(tf.truncated_normal([5,5,1,num_filters1],
                                          stddev=0.1))
h_conv1 = tf.nn.conv2d(x_image, W_conv1,
                       strides=[1,1,1,1], padding='SAME')

b_conv1 = tf.Variable(tf.constant(0.1, shape=[num_filters1]))
h_conv1_cutoff = tf.nn.relu(h_conv1 + b_conv1)

h_pool1 = tf.nn.max_pool(h_conv1_cutoff, ksize=[1,2,2,1],
                         strides=[1,2,2,1], padding='SAME')

**[CNN-04]** 2段目の畳み込みフィルターとプーリング層を定義します。

In [4]:
num_filters2 = 64

W_conv2 = tf.Variable(
            tf.truncated_normal([5,5,num_filters1,num_filters2],
                                stddev=0.1))
h_conv2 = tf.nn.conv2d(h_pool1, W_conv2,
                       strides=[1,1,1,1], padding='SAME')

b_conv2 = tf.Variable(tf.constant(0.1, shape=[num_filters2]))
h_conv2_cutoff = tf.nn.relu(h_conv2 + b_conv2)

h_pool2 = tf.nn.max_pool(h_conv2_cutoff, ksize=[1,2,2,1],
                         strides=[1,2,2,1], padding='SAME')

**[CNN-05]** 全結合層、ドロップアウト層、ソフトマックス関数を定義します。

In [5]:
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*num_filters2])

num_units1 = 7*7*num_filters2
num_units2 = 1024

w2 = tf.Variable(tf.truncated_normal([num_units1, num_units2]))
b2 = tf.Variable(tf.constant(0.1, shape=[num_units2]))
hidden2 = tf.nn.relu(tf.matmul(h_pool2_flat, w2) + b2)

keep_prob = tf.placeholder(tf.float32)
hidden2_drop = tf.nn.dropout(hidden2, keep_prob)

w0 = tf.Variable(tf.zeros([num_units2, 10]))
b0 = tf.Variable(tf.zeros([10]))
p = tf.nn.softmax(tf.matmul(hidden2_drop, w0) + b0)

**[CNN-06]** 誤差関数 loss、トレーニングアルゴリズム train_step、正解率 accuracy を定義します。

In [6]:
t = tf.placeholder(tf.float32, [None, 10])
loss = -tf.reduce_sum(t * tf.log(p))
train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

**[CNN-07]** セッションを用意して、Variable を初期化します。

In [7]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

**[CNN-08]** パラメーターの最適化を20000回繰り返します。

最終的に、テストセットに対して約99%の正解率が得られます。

In [12]:
i = 0
for _ in range(20000):
    i += 1
    batch_xs, batch_ts = mnist.train.next_batch(50)
    sess.run(train_step,
             feed_dict={x:batch_xs, t:batch_ts, keep_prob:0.5})
    if i % 100 == 0:
        loss_vals, acc_vals = [], []
        for c in range(4):
            start = int(len(mnist.test.labels) / 4 * c)
            end = int(len(mnist.test.labels) / 4 * (c+1))
            loss_val, acc_val = sess.run([loss, accuracy],
                feed_dict={x:mnist.test.images[start:end],
                           t:mnist.test.labels[start:end],
                           keep_prob:1.0})
            loss_vals.append(loss_val)
            acc_vals.append(acc_val)
        loss_val = np.sum(loss_vals)
        acc_val = np.mean(acc_vals)
        print ('Step: %d, Loss: %f, Accuracy: %f'
               % (i, loss_val, acc_val))
        saver.save(sess, 'cnn_session', global_step=i)

Step: 100, Loss: 795.472656, Accuracy: 0.975200
Step: 200, Loss: 784.871582, Accuracy: 0.975100
Step: 300, Loss: 768.992859, Accuracy: 0.974800
Step: 400, Loss: 741.119080, Accuracy: 0.975100
Step: 500, Loss: 684.308594, Accuracy: 0.979000
Step: 600, Loss: 680.701233, Accuracy: 0.977600
Step: 700, Loss: 648.459290, Accuracy: 0.978500
Step: 800, Loss: 659.012085, Accuracy: 0.978400
Step: 900, Loss: 659.059326, Accuracy: 0.979800
Step: 1000, Loss: 625.397766, Accuracy: 0.980200
Step: 1100, Loss: 617.125854, Accuracy: 0.981300
Step: 1200, Loss: 604.904480, Accuracy: 0.979600
Step: 1300, Loss: 581.224487, Accuracy: 0.981800
Step: 1400, Loss: 570.867859, Accuracy: 0.980700
Step: 1500, Loss: 578.693481, Accuracy: 0.981100
Step: 1600, Loss: 537.275269, Accuracy: 0.982000
Step: 1700, Loss: 586.680481, Accuracy: 0.981500
Step: 1800, Loss: 505.982300, Accuracy: 0.983400
Step: 1900, Loss: 498.120667, Accuracy: 0.983100
Step: 2000, Loss: 501.746033, Accuracy: 0.983600
Step: 2100, Loss: 493.535461,

KeyboardInterrupt: 

**[CNN-09]** セッション情報を保存したファイルが生成されていることを確認します。

In [13]:
!ls cnn_session*

cnn_session-100.data-00000-of-00001   cnn_session-2300.meta
cnn_session-100.index		      cnn_session-2400.data-00000-of-00001
cnn_session-100.meta		      cnn_session-2400.index
cnn_session-1000.data-00000-of-00001  cnn_session-2400.meta
cnn_session-1000.index		      cnn_session-2500.data-00000-of-00001
cnn_session-1000.meta		      cnn_session-2500.index
cnn_session-1100.data-00000-of-00001  cnn_session-2500.meta
cnn_session-1100.index		      cnn_session-2600.data-00000-of-00001
cnn_session-1100.meta		      cnn_session-2600.index
cnn_session-1200.data-00000-of-00001  cnn_session-2600.meta
cnn_session-1200.index		      cnn_session-2700.data-00000-of-00001
cnn_session-1200.meta		      cnn_session-2700.index
cnn_session-1300.data-00000-of-00001  cnn_session-2700.meta
cnn_session-1300.index		      cnn_session-2800.data-00000-of-00001
cnn_session-1300.meta		      cnn_session-2800.index
cnn_session-1400.data-00000-of-00001  cnn_session-2800.meta
cnn_session-1400.index		      cnn_session-2900.